**Описание скрипта**

В скрипте объединяются данные Рпл / Рзаб от НТЦ и ШТР по нефтяным скважинам.

В итоговой выгрузке в строке, напр. 01.05.2017, указаны давления от НТЦ (Рпл и Рзаб) из строки 01.05.2017, а данные из ШТР взяты из строки 01.04.2017 (предыдущий месяц).

Считаем, что данные приведены на начало месяца!

Категориальные переменные взяты как первое значение за нужный месяц, количественные - как среднее арифметическое за нужный месяц.

Если день в дате в файле от НТЦ не равен 1, то давление приводится на начало текущего (день <= 15) / следующего месяца (день > 15)

In [1]:
import pandas as pd
import numpy as np
import os
import datetime

from pandas.tseries.offsets import DateOffset #для добавления месяцев к дате
from typing import Union
from dotenv import load_dotenv

### 1. Загрузка исходных данных

In [2]:
load_dotenv()
# Устанавливаем настройку для отображения всех столбцов датафрейма
pd.set_option('display.max_columns', None)

os.getenv('FLAG')

'1'

#### 1.1 Загрузка выгрузки из ШТР

In [ ]:
if int(os.getenv('FLAG')) == 0:
    # Загружаем данные из ШТР за все года
    # В выгрузке дебиты протянуты во все года
    from_shtr = pd.read_csv(os.getenv('SHTR_FILE'), encoding='cp1251', sep=';', 
                            parse_dates=['measure_date'], dayfirst=True)
else:
    from_shtr_old = pd.read_csv(os.getenv('SHTR_FILE'), encoding='cp1251', sep=';', 
                            parse_dates=['measure_date'], dayfirst=True)
    from_shtr_new = pd.read_csv(os.getenv('SHTR_FILE_NEW'), encoding='cp1251', sep=';', 
                            parse_dates=['measure_date'], dayfirst=True)
    from_shtr = pd.concat([from_shtr_old, from_shtr_new])
    
    print(f'\n{from_shtr_old.shape[0]} - количество строк в ШТР за весь период разработки')
    print(f'{from_shtr_new.shape[0]} - количество строк в ШТР за последний период')
    print(f'{from_shtr.shape[0]} - количество строк в объединенной выгрузке ШТР \n')
from_shtr.info()

In [6]:
# Проверяем последнюю дату в выгрузке
from_shtr.tail()

,measure_date,well_name,well_id,rig,character,oper_mode,exploitation_name,equipment_name,worktime,downtime,state,t_work_tm,f_rotat_tm,f_rotat,q_fluid,q_fluid_tm,q_oil,wcutting,q_gaz,q_gaz_tm,p_line,p_buff,p_buff_tm,p_bottomhole_research,p_intake_accel,p_intake_accel_tm,gas_factor,p_layer,p_annular,p_annular_tm,p_ag_line,field,p_line_tm,q_methanol_day,choke_d,p_bottom_from_intake,liquid_flow_calc_tm,valve_opening,isInControlFund,equipment_depth
119047,2024-08-11,5216,500521600,11,Нефтяная,NaN,Фонтанный,Фонтанный лифт с ТМСП,24.0,0.0,В работе,0.0,0.0,NaN,87.0,87.0,70.47,3.8,26749.0,26749.0,8.9,38.4,38.4,NaN,87.58,87.58,379.58,NaN,73.61,73.61,8.30,Новопортовское,8.9,NaN,NaN,88.08,104.58,NaN,Нет,2153.03
119048,2024-08-12,5216,500521600,11,Нефтяная,NaN,Фонтанный,Фонтанный лифт с ТМСП,24.0,0.0,В работе,0.0,0.0,NaN,93.0,93.0,75.33,3.8,26454.0,26454.0,8.9,38.4,38.4,NaN,87.56,87.56,351.17,NaN,73.93,73.93,8.29,Новопортовское,8.9,NaN,NaN,88.06,84.46,NaN,Нет,2153.03
119049,2024-08-13,5216,500521600,11,Нефтяная,NaN,Фонтанный,Фонтанный лифт с ТМСП,24.0,0.0,В работе,0.0,0.0,NaN,92.0,92.0,74.52,3.8,26071.0,26071.0,9.2,41.1,41.1,NaN,88.07,88.07,349.85,NaN,73.93,73.93,8.37,Новопортовское,9.2,NaN,NaN,88.57,90.16,NaN,Нет,2153.03
119050,2024-08-14,5216,500521600,11,Нефтяная,NaN,Фонтанный,Фонтанный лифт с ТМСП,24.0,0.0,В работе,0.0,0.0,NaN,89.0,89.0,72.09,3.8,26328.0,26328.0,9.3,41.4,41.4,NaN,87.85,87.85,365.21,NaN,73.61,73.61,8.34,Новопортовское,9.3,NaN,NaN,88.35,88.59,NaN,Нет,2153.03
119051,2024-08-15,5216,500521600,11,Нефтяная,NaN,Фонтанный,Фонтанный лифт с ТМСП,24.0,0.0,В работе,0.0,0.0,NaN,90.0,90.0,72.90,3.8,26728.0,26728.0,8.8,41.4,32.5,NaN,87.60,87.60,366.64,NaN,73.29,73.29,8.18,Новопортовское,8.8,NaN,NaN,88.10,99.33,NaN,Нет,2153.03


#### 1.2 Загрузка пластовых давлений из файлов от НТЦ

In [7]:
'''Из списка ниже выбрать нужный файл, чтобы работать с нужным файлом Excel и 
сохранить графики в нужную папку'''

pressure_file_list = [ 
    'MBALnp1',
    'MBALnp1_0',
    'MBALnp23',
    'MBALnp4',
    'MBALnp51',
    'MBALnp7',
    'MBALnp8',
    'MBALu26',
    'MBALtp14'
]

In [8]:
all_pres_data = []
# Из выбранных файлов с давлениями подгружаем данные
for file in pressure_file_list:      
    fact_pres_data = pd.read_excel(
        os.getenv(r'PRESSURE_PATH').format(file), 
        sheet_name='добыча и Рпл', 
        header=6, 
        parse_dates=['Дата'],
        converters={'NSKV':str}
    )  
    # В список добавляем давления из разных скважин, удаляем пустые строки в столбце со скв.
    all_pres_data.append(fact_pres_data[fact_pres_data['NSKV'].notna()])
    
# Формируем датасет по всем скважинам выбранных пластов
all_pres_data = pd.concat(all_pres_data)
all_pres_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50824 entries, 0 to 445
Data columns (total 62 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NSKV                               50824 non-null  object        
 1   Дата                               50824 non-null  datetime64[ns]
 2   Фонд                               45878 non-null  object        
 3   штуцер                             9234 non-null   float64       
 4   Частота                            466 non-null    object        
 5   обводн., %                         38876 non-null  float64       
 6   Дебит жидкости, т/сут              62 non-null     float64       
 7   Рзаб(нефт)                         7484 non-null   float64       
 8   Прием., м3/сут                     5746 non-null   object        
 9   Рзаб(ППД)                          5539 non-null   float64       
 10  Рпл расч                           4

In [9]:
all_pres_data.to_csv('all_pres_data.csv', sep=';', encoding='cp1251',index=False)

### 2. Обработка

#### 2.1 Преобразование типов данных, снижение объема используемой памяти

In [135]:
# Выведем более подробную статистику и более точные сведения об использовании памяти
from_shtr.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2539408 entries, 0 to 119051
Data columns (total 40 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   measure_date           datetime64[ns]
 1   well_name              object        
 2   well_id                int64         
 3   rig                    object        
 4   character              object        
 5   oper_mode              float64       
 6   exploitation_name      object        
 7   equipment_name         object        
 8   worktime               float64       
 9   downtime               float64       
 10  state                  object        
 11  t_work_tm              float64       
 12  f_rotat_tm             float64       
 13  f_rotat                float64       
 14  q_fluid                float64       
 15  q_fluid_tm             float64       
 16  q_oil                  float64       
 17  wcutting               float64       
 18  q_gaz                  

In [137]:
# Функция для определения объема используемой памяти
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # исходим из предположения о том, что если это не DataFrame, то это Series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)

# В файле с Рпл НП4 есть даты в виде числе и пустые строки в датах. 
# Для обработки таких значений нужна эта функция
def parse_excel_date(excel_date: any) -> Union[datetime.datetime, None]:
    '''Функция для парсинга даты из excel, выполняется обработка различных ошибок'''

    if not excel_date or isinstance(excel_date, datetime.datetime):
        return excel_date

    elif type(excel_date) == int or excel_date.isdigit():
        return datetime.datetime.fromordinal(datetime.datetime(1900, 1, 1).toordinal() + int(excel_date) - 2)
    
    else:
        return pd.to_datetime(excel_date, format='%Y-%m-%d')

In [138]:
# Создаем новый датафрейм для преобразованных данных 
from_shtr_good = from_shtr.copy()

# Находим столбцы с типом данных float (результат - словарь)
float_col = from_shtr_good.select_dtypes(include=['float']).dtypes

# Преобразуем столбцы с типом данных float64 в float32
for col in float_col.keys():
    from_shtr_good[col] = from_shtr_good[col].astype('float32')

# Определим количество используемой памяти
mem_usage(from_shtr_good)

'1845.02 MB'

In [139]:
# Находим столбцы с типом данных object (результат - словарь)
object_col = from_shtr_good.select_dtypes(include=['object']).dtypes
# Уберем номера скв. из названий столбцов
object_col = object_col[1:]

# Преобразуем столбцы с типом данных object в category
for col in object_col.keys():
    from_shtr_good[col] = from_shtr_good[col].astype('category')

# Для объединения с таблицей НТЦ преобразуем имена скв. в строковый тип
from_shtr_good['well_name'] = from_shtr_good['well_name'].astype('str')

# Проверка наличия значений типа inf в датафрейме (появляется при нисходящем изменении типов)
print('Количество значений типа inf =',
from_shtr_good.isin([np.inf, -np.inf]).sum().sum()
)

from_shtr_good.info(memory_usage='deep')

Количество значений типа inf = 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2539408 entries, 0 to 119051
Data columns (total 40 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   measure_date           datetime64[ns]
 1   well_name              object        
 2   well_id                int64         
 3   rig                    category      
 4   character              category      
 5   oper_mode              float32       
 6   exploitation_name      category      
 7   equipment_name         category      
 8   worktime               float32       
 9   downtime               float32       
 10  state                  category      
 11  t_work_tm              float32       
 12  f_rotat_tm             float32       
 13  f_rotat                float32       
 14  q_fluid                float32       
 15  q_fluid_tm             float32       
 16  q_oil                  float32       
 17  wcutting               float32   

In [140]:
all_pres_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50824 entries, 0 to 445
Data columns (total 62 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NSKV                               50824 non-null  object        
 1   Дата                               50824 non-null  datetime64[ns]
 2   Фонд                               45878 non-null  object        
 3   штуцер                             9234 non-null   float64       
 4   Частота                            466 non-null    object        
 5   обводн., %                         38876 non-null  float64       
 6   Дебит жидкости, т/сут              62 non-null     float64       
 7   Рзаб(нефт)                         7484 non-null   float64       
 8   Прием., м3/сут                     5746 non-null   object        
 9   Рзаб(ППД)                          5539 non-null   float64       
 10  Рпл расч                           4

In [141]:
# Готовим таблицу с давлениями для объединения с выгрузкой из ШТР
all_pres_data_1 = all_pres_data.copy()
all_pres_data_1['NSKV'] = all_pres_data_1['NSKV'].astype('str')

# Преобразуем столбец с датами в тип datetime
all_pres_data_1['Дата'] =  all_pres_data_1['Дата'].apply(lambda x: parse_excel_date(x))
all_pres_data_1.head(2)

,NSKV,Дата,Фонд,штуцер,Частота,"обводн., %","Дебит жидкости, т/сут",Рзаб(нефт),"Прием., м3/сут",Рзаб(ППД),Рпл расч,Рпл ГДИС,Рзб ГДИС,Prim,EKSPLDATE,BEGDATE,INJDATE,LSTDATE,Remont,"Плотность, г/см3",Пласты,ГРП,Разбиение на части,"ГФ, м3/т",Unnamed: 24,Рпл model или map_old,D_Рпл_мод,D_Pтек-Рмодель,Тренд Рпл(расч) через 3 мес,Скв,Дата.1,Депрессия,Кпрод,Карта (признак),К-т А,К-т В,имя2,NSKV.1,Рпл (тренд),Рпл (по тренду),Рпл (по Кпрод old),Кпрод (протяжка),"Пл-сть (расч); по % обв, г/см3",Pтмс,dP,Pз (ВНК),Unnamed: 46,dP= АО ВНК-АО в.д. перф,"Дебит жидкости, м3/сут","Пл-сть смеси (ср. по ств.), г/см3",0,NaN,"Дебит жидк/газа, м3/сут",Рзаб(доб),LSTDATE / Рбуф,Ячейки/Пласты,ГРП / ссылки на ГеоБД,Ячейки,Альтитуда,Рбуф,Рзаб (пересчет на пл-сть AU),ГРП / Рзаб
0,1003,2021-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179.3,NaN,ВНР(нов)=179.3 (0 дн; Рпр(глуш)),NaT,NaT,NaN,00:00:00,Ввод новых ГС,1.080,НП 1,http://10.69.167.37:8888/Php_sam/Delo_Skv/Davl...,VNR 1,NaN,http://10.69.167.37:8888/Php_sam/Delo_Skv/Davl...,NaN,NaN,NaN,NaN,1003,2021-05-05 00:00:00,0.0,0.0,0.0,0.0,0.0,1003,1003.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003,2021-06-10,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179.3,NaN,NaN,5/5/21 ВНР(нов)=179 (0дн;Рпр(р-р)),2021-05-06 00:00:00,2021-05-01 00:00:00,00:00:00,2021-06-01 00:00:00,06/05/21Ввод новых ГС (),0.496,НП 1,00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,1003,2021-06-10 00:00:00,0.0,0.0,0.0,0.0,0.0,1003,1003.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Объем используемой памяти для датафрейма ШТР снижен в несколько раз путем преобразования в подтипы, которые используют меньшее число байт для представления каждого значения.

Объем используемой памяти для датафрейма с Рпл изначально небольшой.

Названия скважин в обоих датафреймах преобразованы в строковый тип, даты - в тип дат.

#### 2.2 Объединение ШТР и Рпл

In [142]:
# Проверка отсутствующих скв.
def get_not_found_wells(well_list: pd.Series, well_list_search: pd.Series) -> None:
    '''Ищем отсутствующие скважины.
    well_list:          pd.Series - список со скважинами, которые будем искать
    well_list_search:   pd.Series - список, в котором будем искать скважины
    '''

    not_found_wells = []
    for well in well_list:
        if well not in well_list_search:
            not_found_wells.append(well)

    return not_found_wells

In [143]:
print('Количество скважин в таблице с Рпл равно', len(all_pres_data_1['NSKV'].unique()))
print(f'Количество скважин в таблице ШТР равно', len(from_shtr_good['well_name'].unique()))  

Количество скважин в таблице с Рпл равно 789
Количество скважин в таблице ШТР равно 853


В датафрейме all_fact_pres_data_good давления чаще всего указаны на первое число месяца.

Значит дебиты нужно брать как среднее арифметическое предыдущего месяца.

Напр. если Рпл и Рзаб указаны в дату 01.04.2024, то дебиты берем за все сутки 03.2024.

Следовательно, перед объединением датафреймов с дебитами и давлениями НТЦ, нужно сдвинуть дату в дебитах на месяц вперед.

При этом дебит жидкости в файле НТЦ берется за последние сутки предыдущего месяца.

In [144]:
# Убираем лишние столбцы (не используются в скрипте Айнура)
from_shtr_good = from_shtr_good.loc[:, 'measure_date':'p_bottom_from_intake'].drop(['well_id', 'oper_mode'], axis=1)

# Для объединения датафреймов сдвинем дату на месяц вперед
from_shtr_good['measure_date_new'] = from_shtr_good['measure_date'] + DateOffset(months=1)

# Для объединения датафреймов создаем столбцы с месяцем и годом
from_shtr_good['month'] = pd.DatetimeIndex(from_shtr_good['measure_date_new']).month
from_shtr_good['year'] = pd.DatetimeIndex(from_shtr_good['measure_date_new']).year

from_shtr_good.head()

,measure_date,well_name,rig,character,exploitation_name,equipment_name,worktime,downtime,state,t_work_tm,f_rotat_tm,f_rotat,q_fluid,q_fluid_tm,q_oil,wcutting,q_gaz,q_gaz_tm,p_line,p_buff,p_buff_tm,p_bottomhole_research,p_intake_accel,p_intake_accel_tm,gas_factor,p_layer,p_annular,p_annular_tm,p_ag_line,field,p_line_tm,q_methanol_day,choke_d,p_bottom_from_intake,measure_date_new,month,year
0,2024-04-01,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,В работе,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,18.6,0.0,79.879997,79.879997,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.1,NaN,0.0,86.57,2024-05-01,5,2024
1,2024-04-02,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,В работе,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,17.5,0.0,79.879997,79.709999,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.5,NaN,0.0,86.57,2024-05-02,5,2024
2,2024-04-03,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,В работе,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,17.5,0.0,79.879997,79.709999,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.5,NaN,0.0,86.57,2024-05-03,5,2024
3,2024-04-04,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,NaN,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,17.5,0.0,79.879997,79.709999,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.5,NaN,0.0,86.57,2024-05-04,5,2024
4,2024-04-05,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,NaN,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,17.5,0.0,79.879997,79.709999,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.5,NaN,0.0,86.57,2024-05-05,5,2024


In [146]:
# Переименуем столбцы в датафрейме с давлениями от НТЦ
all_pres_data_1.rename(columns={
    'NSKV': 'well_name', 'Дата': 'measure_date', 'Рзаб(нефт)': 'bhp1',
    'Рзаб(доб)': 'bhp2', 'Рзаб(ППД)': 'bhp_inj',
    'Рпл расч':'layer_pres_calc', 'Рпл ГДИС': 'layer_pres_research'
    }, inplace=True)

# Оставляем в датафрейме только нужные столбцы
all_pres_data_1 = all_pres_data_1[
    ['well_name', 'measure_date', 'bhp1', 'bhp2', 'bhp_inj', 'layer_pres_calc', 'layer_pres_research']
].reset_index(drop=True)

In [147]:
def get_round_date(date: any) -> any:
    ''' Функция формирует дату с учетом округления дней.
      Если день относится ко второй половине месяца, то округляет день до 1 числа 
      следующего месяца. 
      Иначе округляет день до 1 числа текущего месяца'''
    if  date.day >= 15:
        return date + pd.offsets.MonthBegin()

    # Если день относится к первой половине месяца, то округлим день до 1 этого месяца
    return date + pd.offsets.MonthBegin() - DateOffset(months=1)

Округлим даты в таблице с давлениями на первое число месяца

In [148]:
all_pres_data_1['measure_date'] = all_pres_data_1['measure_date'].apply(get_round_date)
all_pres_data_1.head()

,well_name,measure_date,bhp1,bhp2,bhp_inj,layer_pres_calc,layer_pres_research
0,1003,2021-05-01,NaN,NaN,NaN,NaN,179.3
1,1003,2021-06-01,NaN,NaN,NaN,179.3,NaN
2,1003,2021-07-01,147.0,NaN,NaN,179.3,NaN
3,1003,2021-07-01,140.0,NaN,NaN,175.0,NaN
4,1003,2021-08-01,133.9,NaN,NaN,171.1,NaN


Даты в датафрейме с давлениями от НТЦ преобразованы корректно, заменим значения в исходном столбце

In [149]:
# Найдем задвоения дат по скв.
print('Количество задвоений дат по скважине =',
    all_pres_data_1[['well_name', 'measure_date']].duplicated().sum()
)

# Для объединения датафреймов создаем столбцы с месяцем и годом
all_pres_data_1['month'] = pd.DatetimeIndex(all_pres_data_1['measure_date']).month
all_pres_data_1['year'] = pd.DatetimeIndex(all_pres_data_1['measure_date']).year

Количество задвоений дат по скважине = 11985


In [150]:
try:
    # Заменяем английские буквы Р и русские буквы Р в названии скв.
    all_pres_data_1['well_name'] = all_pres_data_1['well_name'].str.replace('Р', '')
    all_pres_data_1['well_name'] = all_pres_data_1['well_name'].str.replace('P', '')
    all_pres_data_1['well_name'] = all_pres_data_1['well_name'].str.replace('R', '')
    from_shtr_good['well_name'] = from_shtr_good['well_name'].str.replace('Р', '')
    from_shtr_good['well_name'] = from_shtr_good['well_name'].str.replace('P', '')
    from_shtr_good['well_name'] = from_shtr_good['well_name'].str.replace('R', '')    
    
    # Проверяем есть ли все скв. из одного датафрейма в другом
    not_found_wells = get_not_found_wells(
        all_pres_data_1['well_name'].unique(), 
        from_shtr_good['well_name'].unique()
    )

    if not_found_wells:
        print(f'Not found wells: {len(not_found_wells)}')
        print(not_found_wells)
    else:
        print('success!')

except KeyError:
    print('Английские буквы P заменены на русские в названии всех скважин')

Not found wells: 12
['3402', '3404', '3405', '3406', '3407', '3408', '3409', '3410', '3411', '3412', '3501', '8058']


In [151]:
# После объединения разных файлов с давлениями устанавливаем уникальные индексы
all_pres_data_1 = all_pres_data_1.reset_index()

# Найдем значение, из-за которого не получается преобразовать тип столбца 'layer_pres_research'
all_pres_data_1['layer_pres_research'].value_counts().sort_values()

74.0        1
117.89      1
127.3       1
86.7        1
179.44      1
         ... 
184        23
205        23
184.5      23
175        28
          900
Name: layer_pres_research, Length: 1757, dtype: int64

In [152]:
# Избавляемся от ненужных символов и преобразуем в тип float (обязательно нужен для сводной таблицы)
all_pres_data_1['layer_pres_calc'] = (all_pres_data_1['layer_pres_calc'].replace(' ', np.nan)
                                      .astype('float')
                                      )
all_pres_data_1['layer_pres_research'] = (all_pres_data_1['layer_pres_research']
                                          .replace(' ', np.nan)
                                          .astype('float')
                                         )

# Создаем новый датафрейм, т.к. перед удалением строк нужно оставить подготовленный датафрейм
all_pres_data_good = all_pres_data_1.copy()

# Вычислим средние значения давлений за каждый месяц
all_pres_data_good = all_pres_data_good.groupby(['well_name', 'year', 'month']).agg(
    {'measure_date': 'first','bhp1': 'mean', 'bhp2': 'mean', 'bhp_inj': 'mean', 
     'layer_pres_calc': 'mean', 'layer_pres_research': 'mean'}
    ).reset_index()
all_pres_data_good.head()

,well_name,year,month,measure_date,bhp1,bhp2,bhp_inj,layer_pres_calc,layer_pres_research
0,1003,2021,5,2021-05-01,NaN,NaN,NaN,NaN,179.3
1,1003,2021,6,2021-06-01,NaN,NaN,NaN,179.30,NaN
2,1003,2021,7,2021-07-01,143.5,NaN,NaN,177.15,NaN
3,1003,2021,8,2021-08-01,133.9,NaN,NaN,171.10,NaN
4,1003,2021,9,2021-09-01,132.4,NaN,NaN,167.59,NaN


In [153]:
# Бежим по строкам и записываем данные в ячейку
for idx, row in all_pres_data_good.iterrows():

    # Объединяем столбцы с Рзаб для доб. скв.
    if all_pres_data_good.loc[idx, 'bhp1'] > 0:
        all_pres_data_good.loc[idx, 'bhp'] = all_pres_data_good.loc[idx, 'bhp1']
        all_pres_data_good.loc[idx, 'character'] = 'Нефтяная'

    elif all_pres_data_good.loc[idx, 'bhp2'] > 0:
        all_pres_data_good.loc[idx, 'bhp'] = all_pres_data_good.loc[idx, 'bhp2']
        all_pres_data_good.loc[idx, 'character'] = 'Нефтяная'

    # Если скв. в ППД, то запишем Рзаб в столбец 'bhp'
    elif all_pres_data_good.loc[idx, 'bhp_inj'] > 0:
        if np.isnan(all_pres_data_good.loc[idx, 'bhp']):
            all_pres_data_good.loc[idx, 'bhp'] = all_pres_data_good.loc[idx, 'bhp_inj']
            all_pres_data_good.loc[idx, 'character'] = 'Нагнетательная'
    
    else:
        all_pres_data_good.loc[idx, 'bhp'] = np.nan
        all_pres_data_good.loc[idx, 'character'] = np.nan

# Проставим назначение газовых скважин
for idx, row in all_pres_data_good.iterrows():
    
    if all_pres_data_good.loc[idx, 'well_name'][0] == '3':
        all_pres_data_good.loc[idx, 'character'] = 'Газовая'

all_pres_data_good['character'].value_counts()

Нефтяная          27819
Нагнетательная     4560
Газовая            2467
Name: character, dtype: int64

Удалим строки с газовыми и нагнетательными скв.

In [154]:
all_pres_data_good = all_pres_data_good.query('character == "Нефтяная"')
all_pres_data_good.head()

,well_name,year,month,measure_date,bhp1,bhp2,bhp_inj,layer_pres_calc,layer_pres_research,bhp,character
2,1003,2021,7,2021-07-01,143.5,NaN,NaN,177.15,NaN,143.5,Нефтяная
3,1003,2021,8,2021-08-01,133.9,NaN,NaN,171.10,NaN,133.9,Нефтяная
4,1003,2021,9,2021-09-01,132.4,NaN,NaN,167.59,NaN,132.4,Нефтяная
5,1003,2021,10,2021-10-01,130.4,NaN,NaN,167.00,NaN,130.4,Нефтяная
6,1003,2021,11,2021-11-01,125.0,NaN,NaN,165.30,NaN,125.0,Нефтяная


In [155]:
# Оставляем только один итоговый столбец с Рзаб
all_pres_data_good['bhp1'] = all_pres_data_good['bhp']
all_pres_data_good.drop(columns=['measure_date', 'bhp', 'bhp2', 'bhp_inj'], inplace=True)
all_pres_data_good.rename(columns={'bhp1': 'bhp'}, inplace=True)
display(all_pres_data_good.head())
all_pres_data_good.info()

,well_name,year,month,bhp,layer_pres_calc,layer_pres_research,character
2,1003,2021,7,143.5,177.15,NaN,Нефтяная
3,1003,2021,8,133.9,171.10,NaN,Нефтяная
4,1003,2021,9,132.4,167.59,NaN,Нефтяная
5,1003,2021,10,130.4,167.00,NaN,Нефтяная
6,1003,2021,11,125.0,165.30,NaN,Нефтяная


<class 'pandas.core.frame.DataFrame'>
Int64Index: 27819 entries, 2 to 38838
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   well_name            27819 non-null  object 
 1   year                 27819 non-null  int64  
 2   month                27819 non-null  int64  
 3   bhp                  27819 non-null  float64
 4   layer_pres_calc      27573 non-null  float64
 5   layer_pres_research  2227 non-null   float64
 6   character            27819 non-null  object 
dtypes: float64(3), int64(2), object(2)
memory usage: 1.7+ MB


 Формируем объединенный датафрейм ШТР и НТЦ

In [156]:
# Оставляем только даты, в которых указано Рпл, рассчитанное НТЦ
merged_df = from_shtr_good.merge(all_pres_data_good, 
                                 on=['well_name', 'month', 'year'], 
                                 how='left')

display(merged_df.head())
display(merged_df.info(memory_usage='deep'))

,measure_date,well_name,rig,character_x,exploitation_name,equipment_name,worktime,downtime,state,t_work_tm,f_rotat_tm,f_rotat,q_fluid,q_fluid_tm,q_oil,wcutting,q_gaz,q_gaz_tm,p_line,p_buff,p_buff_tm,p_bottomhole_research,p_intake_accel,p_intake_accel_tm,gas_factor,p_layer,p_annular,p_annular_tm,p_ag_line,field,p_line_tm,q_methanol_day,choke_d,p_bottom_from_intake,measure_date_new,month,year,bhp,layer_pres_calc,layer_pres_research,character_y
0,2024-04-01,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,В работе,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,18.6,0.0,79.879997,79.879997,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.1,NaN,0.0,86.57,2024-05-01,5,2024,91.7,110.3,NaN,Нефтяная
1,2024-04-02,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,В работе,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,17.5,0.0,79.879997,79.709999,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.5,NaN,0.0,86.57,2024-05-02,5,2024,91.7,110.3,NaN,Нефтяная
2,2024-04-03,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,В работе,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,17.5,0.0,79.879997,79.709999,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.5,NaN,0.0,86.57,2024-05-03,5,2024,91.7,110.3,NaN,Нефтяная
3,2024-04-04,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,NaN,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,17.5,0.0,79.879997,79.709999,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.5,NaN,0.0,86.57,2024-05-04,5,2024,91.7,110.3,NaN,Нефтяная
4,2024-04-05,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,0.0,NaN,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,17.5,0.0,79.879997,79.709999,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.5,NaN,0.0,86.57,2024-05-05,5,2024,91.7,110.3,NaN,Нефтяная


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2539408 entries, 0 to 2539407
Data columns (total 41 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   measure_date           datetime64[ns]
 1   well_name              object        
 2   rig                    category      
 3   character_x            category      
 4   exploitation_name      category      
 5   equipment_name         category      
 6   worktime               float32       
 7   downtime               float32       
 8   state                  category      
 9   t_work_tm              float32       
 10  f_rotat_tm             float32       
 11  f_rotat                float32       
 12  q_fluid                float32       
 13  q_fluid_tm             float32       
 14  q_oil                  float32       
 15  wcutting               float32       
 16  q_gaz                  float32       
 17  q_gaz_tm               float32       
 18  p_line                

None

#### 2.3 Обработка объединенной таблицы

##### Заполнение пропущенных значений

In [157]:
def set_q_fluid(row: pd.Series) -> Union[None, float]:
    '''Устанавливаем по строкам дебит жидкости по телеметрии'''

    if not np.isnan(row['q_fluid']):
        return row['q_fluid']

    if row['q_fluid_tm'] > 11 and row['state'] != 'Остановлена':
        return row['q_fluid_tm']
    
    return np.nan

def set_q_gaz(row: pd.Series) -> Union[None, float]:
    '''Устанавливаем по строкам дебит газа по телеметрии.'''

    if not np.isnan(row['q_gaz']):
        return row['q_gaz']

    if row['q_gaz_tm'] > 100 and row['state'] != 'Остановлена':
        return row['q_gaz_tm']
    
    return np.nan  

# Если значения в столбце p_buff отсутствуют или меньше  давления на ЦПС (7), 
# то берем значения из столбца p_buff_tm
def set_p_buff(row: pd.Series) -> Union[None, float]:
    '''Устанавливаем по строкам буферное давление по телеметрии.'''

    if not np.isnan(row['p_buff']):
        return row['p_buff']

    if row['p_buff'] < 7 and row['p_buff_tm'] > 0 and row['state'] != 'Остановлена':
        return row['p_buff_tm']
    
    return np.nan  

def set_p_intake(row: pd.Series) -> Union[None, float]:
    '''Устанавливаем по строкам давление на приеме ЭЦН по телеметрии.'''

    if not np.isnan(row['p_intake_accel']):
        return row['p_intake_accel']

    if row['p_intake_accel_tm'] > 0 and row['state'] != 'Остановлена':
        return row['p_intake_accel_tm']
    
    return np.nan

def set_p_line(row: pd.Series) -> Union[None, float]:
    '''Устанавливаем по строкам линейное давление по телеметрии.'''

    if not np.isnan(row['p_line']):
        return row['p_line']

    if row['p_line_tm'] > 0 and row['state'] != 'Остановлена':
        return row['p_line_tm']
    
    if np.isnan(row['p_line_tm']) and row['state'] != 'Остановлена':
        return row['p_ag_line']
    
    return np.nan

In [158]:
# Подсчитываем кол-во значений по каждому состоянию скв.
merged_df.groupby('state')['state'].count()

state
В бездействии                             29021
В консервации                             56785
В ликвидации                             286763
В накоплении/под циклической закачкой      3837
В ожидании освоения прошлых лет             338
В освоении                                 9449
В работе                                 872697
ВНР                                       10858
Наблюдательная                              881
Остановлена                               45186
Пьезометрическая                          68816
Name: state, dtype: int64

In [160]:
# Оставляем в датафрейме только строки с нужными состояниями скв.
good_merged_df = merged_df.loc[
    (merged_df['state'] == 'В освоении') | (merged_df['state'] == 'В работе') 
    | (merged_df['state'] == 'ВНР') | (merged_df['state'] == 'Остановлена')
    ]

# Ищем пропущенные значения во всех столбцах
good_merged_df.isna().sum()

measure_date                  0
well_name                     0
rig                           0
character_x                6436
exploitation_name          3713
equipment_name            13567
worktime                      0
downtime                      0
state                         0
t_work_tm                222355
f_rotat_tm               194136
f_rotat                   57815
q_fluid                   22675
q_fluid_tm               166719
q_oil                     38147
wcutting                  27463
q_gaz                     27440
q_gaz_tm                 165960
p_line                    13273
p_buff                    13222
p_buff_tm                636345
p_bottomhole_research    541557
p_intake_accel            71753
p_intake_accel_tm        235261
gas_factor                    0
p_layer                   63352
p_annular                 13562
p_annular_tm             634552
p_ag_line                317990
field                         0
p_line_tm                215866
q_methan

In [161]:
# Анализируем, можно ли брать дебит жидкости по телеметрии, если нет значений в столбце q_fluid
good_merged_df_fluid = (good_merged_df.loc[
    (good_merged_df['q_fluid'].isna()) 
    & (good_merged_df['q_fluid_tm'] > 11)
    & (good_merged_df['state'] != 'Остановлена')
                   ]
      [['measure_date', 'well_name', 'state', 'q_fluid', 'q_fluid_tm']]
      .sort_values(by='q_fluid_tm', ascending=True)
)
good_merged_df_fluid

,measure_date,well_name,state,q_fluid,q_fluid_tm
2452054,2024-04-29,2085,ВНР,NaN,12.0
2129365,2023-07-27,9438,ВНР,NaN,13.0
2450166,2024-04-01,4553,В работе,NaN,13.0
1844696,2023-08-27,9107,ВНР,NaN,13.0
2452052,2024-04-27,2085,ВНР,NaN,13.0
...,...,...,...,...,...
2372317,2023-07-10,9832,ВНР,NaN,437.0
2525710,2024-07-09,8014,ВНР,NaN,497.0
2525711,2024-07-10,8014,ВНР,NaN,506.0
2485059,2024-05-03,9656,В работе,NaN,563.0


In [162]:
good_merged_df_fluid.groupby('state').count()

,measure_date,well_name,q_fluid,q_fluid_tm
state,,,,
В бездействии,0,0,0,0
В консервации,0,0,0,0
В ликвидации,0,0,0,0
В накоплении/под циклической закачкой,0,0,0,0
В ожидании освоения прошлых лет,0,0,0,0
В освоении,13,13,0,13
В работе,532,532,0,532
ВНР,869,869,0,869
Наблюдательная,0,0,0,0


Оценим адекватность дебита жидкости по телеметрии в динамике: часто в строках с пропусками в столбце 'q_fluid' значения 'q_fluid_tm' неадекватно низкие.
Попробуем найти минимальное адекватное значение q_fluid_tm, заполним q_fluid из q_fluid_tm.

In [ ]:
display(
    merged_df.loc[
        (merged_df['measure_date'] > '2023-07-20') 
        & (merged_df['well_name'] == '9530')
    ].head()
)

# Заполним q_fluid из q_fluid_tm
good_merged_df['q_fluid'] = good_merged_df.apply(set_q_fluid, axis=1)

# Ищем пропущенные значения в столбце 'q_fluid'
print('Количество пропущенных значений в столбце q_fluid =',
      good_merged_df['q_fluid'].isna().sum()
)

In [164]:
# Анализируем, можно ли брать дебит газа по телеметрии, если нет значений в столбце q_gas
good_merged_df_gas = (good_merged_df.loc[
    (good_merged_df['q_gaz'].isna()) 
    & (good_merged_df['q_gaz_tm'] > 100)
    & (good_merged_df['state'] != 'Остановлена')
                   ]
      [['measure_date', 'well_name', 'state','q_gaz', 'q_gaz_tm']]
      .sort_values(by='q_gaz_tm', ascending=True).reset_index(drop=True)
)
good_merged_df_gas

,measure_date,well_name,state,q_gaz,q_gaz_tm
0,2023-07-12,4560,ВНР,NaN,101.0
1,2024-05-25,9357,ВНР,NaN,101.0
2,2023-12-03,2025,ВНР,NaN,104.0
3,2023-08-26,7062,ВНР,NaN,109.0
4,2023-07-10,2780,ВНР,NaN,112.0
...,...,...,...,...,...
1394,2023-09-08,9510,ВНР,NaN,275641.0
1395,2024-06-29,4561,ВНР,NaN,283728.0
1396,2023-09-09,9510,ВНР,NaN,290003.0
1397,2023-09-10,9510,ВНР,NaN,305256.0


In [165]:
good_merged_df_gas.groupby('state').count()

,measure_date,well_name,q_gaz,q_gaz_tm
state,,,,
В бездействии,0,0,0,0
В консервации,0,0,0,0
В ликвидации,0,0,0,0
В накоплении/под циклической закачкой,0,0,0,0
В ожидании освоения прошлых лет,0,0,0,0
В освоении,0,0,0,0
В работе,572,572,0,572
ВНР,827,827,0,827
Наблюдательная,0,0,0,0


Заполним q_gaz из q_gaz_tm:

In [166]:
display(good_merged_df_gas.loc[
    (good_merged_df_gas['measure_date'] > '2023-07-01') 
    & (good_merged_df_gas['well_name'] == '4144')
    ].head(20))

good_merged_df['q_gaz'] = good_merged_df.apply(set_q_gaz, axis=1)

# Ищем пропущенные значения
print('Количество пропущенных значений в столбце q_fluid =',
      good_merged_df['q_gaz'].isna().sum()
)

In [167]:
# Анализируем, можно ли брать Рбуф по телеметрии, если нет значений в столбце p_buff
good_merged_df_p_buff = (good_merged_df.loc[
    (good_merged_df['p_buff'].isna()) 
    & (good_merged_df['p_buff_tm'] > 0)
    & (good_merged_df['state'] != 'Остановлена')
                   ]
      [['measure_date', 'well_name', 'state','p_buff', 'p_buff_tm']]
      .sort_values(by='well_name', ascending=True).reset_index(drop=True)
)
good_merged_df_p_buff.head(20)

,measure_date,well_name,state,p_buff,p_buff_tm
0,2024-06-01,1502,ВНР,NaN,26.900000
1,2024-06-03,1502,ВНР,NaN,26.600000
2,2024-05-31,1502,ВНР,NaN,26.400000
3,2024-05-30,1502,ВНР,NaN,26.400000
4,2024-06-02,1502,ВНР,NaN,26.400000
5,2023-12-26,1529,ВНР,NaN,102.000000
6,2023-12-27,1529,ВНР,NaN,102.000000
7,2023-12-30,1529,ВНР,NaN,102.000000
8,2023-12-28,1529,ВНР,NaN,102.000000
9,2023-12-29,1529,ВНР,NaN,102.000000


In [168]:
good_merged_df_p_buff.groupby('state').count()

,measure_date,well_name,p_buff,p_buff_tm
state,,,,
В бездействии,0,0,0,0
В консервации,0,0,0,0
В ликвидации,0,0,0,0
В накоплении/под циклической закачкой,0,0,0,0
В ожидании освоения прошлых лет,0,0,0,0
В освоении,0,0,0,0
В работе,73,73,0,73
ВНР,359,359,0,359
Наблюдательная,0,0,0,0


Заполним p_buff из p_buff_tm. Если значения в столбце p_buff отсутствуют или меньше давления на ЦПС, то берем значения из столбца p_buff_tm

In [ ]:
(good_merged_df.loc[
    (good_merged_df['measure_date'] > '2023-08-17') 
    & (good_merged_df['well_name'] == '2086')]
    [['measure_date', 'well_name', 'state','p_buff', 'p_buff_tm']]
    .head(20))
# Заполним p_buff из p_buff_tm
good_merged_df['p_buff'] = good_merged_df.apply(set_p_buff, axis=1)

# Ищем пропущенные значения
print('Количество пропущенных значений в столбце p_buff =',
      good_merged_df['p_buff'].isna().sum()
)

In [170]:
# Анализируем, можно ли брать Рприем по телеметрии, если нет значений в столбце p_intake_accel
good_merged_df_p_intake = (good_merged_df.loc[
    (good_merged_df['p_intake_accel'].isna()) 
    & (good_merged_df['p_intake_accel_tm'] > 0)
    & (good_merged_df['state'] != 'Остановлена')
                   ]
      [['measure_date', 'well_name', 'state','p_intake_accel', 'p_intake_accel_tm']]
      .sort_values(by='well_name', ascending=True).reset_index(drop=True)
)
good_merged_df_p_intake.head(20)

,measure_date,well_name,state,p_intake_accel,p_intake_accel_tm
0,2023-05-07,1004,ВНР,NaN,131.389999
1,2023-05-08,1004,ВНР,NaN,133.220001
2,2023-05-09,1004,ВНР,NaN,123.430000
3,2023-05-10,1004,ВНР,NaN,125.339996
4,2024-04-28,1102,ВНР,NaN,108.800003
5,2024-04-30,1102,ВНР,NaN,109.940002
6,2024-05-01,1102,ВНР,NaN,109.459999
7,2024-04-29,1102,ВНР,NaN,109.639999
8,2024-06-01,1502,ВНР,NaN,51.349998
9,2024-05-30,1502,ВНР,NaN,57.549999


In [171]:
good_merged_df_p_intake.groupby('state').count()

,measure_date,well_name,p_intake_accel,p_intake_accel_tm
state,,,,
В бездействии,0,0,0,0
В консервации,0,0,0,0
В ликвидации,0,0,0,0
В накоплении/под циклической закачкой,0,0,0,0
В ожидании освоения прошлых лет,0,0,0,0
В освоении,978,978,0,978
В работе,6485,6485,0,6485
ВНР,1052,1052,0,1052
Наблюдательная,0,0,0,0


Заполним p_intake_accel из p_intake_accel_tm:

In [ ]:
(good_merged_df.loc[
    (good_merged_df['measure_date'] > '2023-12-08') 
    & (good_merged_df['well_name'] == '2025')]
    [['measure_date', 'well_name', 'state','p_intake_accel', 'p_intake_accel_tm']]
    .head(20))

# Заполним p_intake_accel из p_intake_accel_tm
good_merged_df['p_intake_accel'] = good_merged_df.apply(set_p_intake, axis=1)

# Ищем пропущенные значения
print('Количество пропущенных значений в столбце p_intake_accel =',
      good_merged_df['p_intake_accel'].isna().sum()
)

In [173]:
# Анализируем, можно ли брать Рзатруб по телеметрии, если нет значений в столбце p_annular
good_merged_df_p_annular = (good_merged_df.loc[
    (good_merged_df['p_annular'].isna()) 
    & (good_merged_df['p_annular_tm'] > 0)
    & (good_merged_df['state'] != 'Остановлена')
                   ]
      [['measure_date', 'well_name', 'state','p_annular', 'p_annular_tm']]
      .sort_values(by='p_annular_tm', ascending=True).reset_index(drop=True)
)
good_merged_df_p_annular.head(20)

,measure_date,well_name,state,p_annular,p_annular_tm
0,2023-03-18,9736,ВНР,NaN,10.0
1,2022-12-30,2146D,ВНР,NaN,10.0
2,2023-03-19,9736,ВНР,NaN,10.0
3,2023-03-15,9737,ВНР,NaN,10.0
4,2023-03-16,9737,ВНР,NaN,10.0
5,2023-03-17,9737,ВНР,NaN,10.0
6,2023-03-18,9737,ВНР,NaN,10.0
7,2022-12-30,2136D,ВНР,NaN,10.0
8,2023-03-19,9737,ВНР,NaN,10.0
9,2023-03-18,9379,ВНР,NaN,10.0


In [174]:
good_merged_df_p_annular.groupby('state').count()

,measure_date,well_name,p_annular,p_annular_tm
state,,,,
В бездействии,0,0,0,0
В консервации,0,0,0,0
В ликвидации,0,0,0,0
В накоплении/под циклической закачкой,0,0,0,0
В ожидании освоения прошлых лет,0,0,0,0
В освоении,0,0,0,0
В работе,59,59,0,59
ВНР,349,349,0,349
Наблюдательная,0,0,0,0


In [175]:
display(good_merged_df.loc[
    (good_merged_df['measure_date'] > '2023-03-10') 
    & (good_merged_df['well_name'] == '9716')
    ].head(10))

,measure_date,well_name,rig,character_x,exploitation_name,equipment_name,worktime,downtime,state,t_work_tm,f_rotat_tm,f_rotat,q_fluid,q_fluid_tm,q_oil,wcutting,q_gaz,q_gaz_tm,p_line,p_buff,p_buff_tm,p_bottomhole_research,p_intake_accel,p_intake_accel_tm,gas_factor,p_layer,p_annular,p_annular_tm,p_ag_line,field,p_line_tm,q_methanol_day,choke_d,p_bottom_from_intake,measure_date_new,month,year,bhp,layer_pres_calc,layer_pres_research,character_y
2348344,2023-03-15,9716,23.0,Нефтяная,NaN,NaN,24.0,0.0,ВНР,NaN,35.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.0,166.960007,166.960007,0.000000,201.0,NaN,10.0,33.560001,Новопортовское,28.0,NaN,0.0,NaN,2023-04-15,4,2023,124.0,184.0,203.9,Нефтяная
2348345,2023-03-16,9716,23.0,Нефтяная,Электропогружным насосом,342 МТ5А-320,24.0,0.0,ВНР,NaN,35.0,0.0,293.0,293.0,60.200001,NaN,25418.0,25418.000000,NaN,NaN,10.0,0.0,153.440002,153.440002,0.000000,201.0,NaN,10.0,33.669998,Новопортовское,28.0,NaN,0.0,NaN,2023-04-16,4,2023,124.0,184.0,203.9,Нефтяная
2348346,2023-03-17,9716,23.0,Нефтяная,Электропогружным насосом,342 МТ5А-320,24.0,0.0,ВНР,NaN,35.0,0.0,262.0,262.0,77.209999,NaN,29884.0,29884.000000,NaN,NaN,10.0,0.0,147.050003,147.050003,0.000000,201.0,NaN,10.0,34.240002,Новопортовское,28.0,NaN,0.0,NaN,2023-04-17,4,2023,124.0,184.0,203.9,Нефтяная
2348347,2023-03-18,9716,23.0,Нефтяная,Электропогружным насосом,342 МТ5А-320,24.0,0.0,ВНР,NaN,35.0,0.0,251.0,251.0,73.970001,NaN,31331.0,31331.000000,NaN,NaN,10.0,0.0,142.100006,142.100006,0.000000,201.0,NaN,10.0,34.970001,Новопортовское,28.0,NaN,0.0,NaN,2023-04-18,4,2023,124.0,184.0,203.9,Нефтяная
2348348,2023-03-19,9716,23.0,Нефтяная,Электропогружным насосом,342 МТ5А-320,24.0,0.0,ВНР,NaN,35.0,0.0,247.0,247.0,82.349998,NaN,35764.0,35764.000000,NaN,NaN,10.0,0.0,138.250000,138.250000,0.000000,201.0,NaN,10.0,34.840000,Новопортовское,28.0,NaN,0.0,NaN,2023-04-19,4,2023,124.0,184.0,203.9,Нефтяная
2348349,2023-03-20,9716,23.0,Нефтяная,Электропогружным насосом,342 МТ5А-320,24.0,0.0,ВНР,NaN,35.0,35.0,250.0,247.0,101.040001,52.0,136350.0,35764.000000,28.0,44.0,10.0,0.0,136.800003,135.710007,1349.469971,201.0,112.0,10.0,35.830002,Новопортовское,28.0,NaN,0.0,144.809998,2023-04-20,4,2023,124.0,184.0,203.9,Нефтяная
2348350,2023-03-21,9716,23.0,Нефтяная,Электропогружным насосом,342 МТ5А-320,24.0,0.0,В работе,NaN,35.0,35.0,250.0,247.0,96.830002,54.0,136350.0,35764.000000,28.0,44.0,10.0,0.0,134.800003,134.820007,1408.140015,201.0,112.0,10.0,34.740002,Новопортовское,28.0,NaN,0.0,142.869995,2023-04-21,4,2023,124.0,184.0,203.9,Нефтяная
2348351,2023-03-22,9716,23.0,Нефтяная,Электропогружным насосом,342 МТ5А-320,24.0,0.0,В работе,9.72,34.0,35.0,250.0,225.0,96.830002,54.0,136350.0,73262.007812,36.0,40.0,10.0,0.0,134.800003,122.599998,1408.140015,201.0,130.0,10.0,34.549999,Новопортовское,28.0,NaN,0.0,142.869995,2023-04-22,4,2023,124.0,184.0,203.9,Нефтяная
2348352,2023-03-23,9716,23.0,Нефтяная,Электропогружным насосом,342 МТ5А-320,24.0,0.0,В работе,24.00,35.0,35.0,254.0,254.0,98.379997,54.0,64409.0,64408.718750,36.0,40.0,10.0,0.0,119.800003,119.750000,654.700012,201.0,130.0,10.0,34.520000,Новопортовское,28.0,NaN,0.0,128.169998,2023-04-23,4,2023,124.0,184.0,203.9,Нефтяная
2348353,2023-03-24,9716,23.0,Нефтяная,Электропогружным насосом,342 МТ5А-320,24.0,0.0,В работе,24.00,35.0,35.0,254.0,254.0,98.379997,54.0,64409.0,64408.718750,36.0,41.0,10.0,0.0,117.099998,117.099998,654.700012,201.0,70.0,10.0,34.740002,Новопортовское,28.0,NaN,0.0,125.480003,2023-04-24,4,2023,124.0,184.0,203.9,Нефтяная


Значения в столбце 'p_annular' сильно отличаются от значений 'p_annular_tm' в более поздние даты.

Решено не заполнять пропуски в столбце 'p_annular'.

In [176]:
# Анализируем, можно ли брать Рбуф по телеметрии, если нет значений в столбце p_buff
good_merged_df_p_line = (good_merged_df.loc[
    (good_merged_df['p_line'].isna()) 
    & (good_merged_df['p_line_tm'] > 0)
    & (good_merged_df['state'] != 'Остановлена')
                   ]
      [['measure_date', 'well_name', 'state','p_line', 'p_line_tm', 'p_ag_line']]
      .sort_values(by='well_name', ascending=True).reset_index(drop=True)
)
good_merged_df_p_line.head(20)

,measure_date,well_name,state,p_line,p_line_tm,p_ag_line
0,2024-05-31,1502,ВНР,NaN,23.799999,25.900000
1,2024-06-01,1502,ВНР,NaN,23.500000,25.799999
2,2024-05-30,1502,ВНР,NaN,24.700001,25.969999
3,2024-06-02,1502,ВНР,NaN,23.500000,25.969999
4,2024-06-03,1502,ВНР,NaN,23.500000,25.930000
5,2023-11-01,1524,ВНР,NaN,12.700000,28.850000
6,2023-11-02,1524,ВНР,NaN,27.299999,28.850000
7,2023-11-03,1524,ВНР,NaN,27.600000,28.850000
8,2023-11-04,1524,ВНР,NaN,28.299999,28.850000
9,2024-01-24,1524,ВНР,NaN,28.100000,26.440001


In [177]:
good_merged_df_p_line.groupby('state').count()

,measure_date,well_name,p_line,p_line_tm,p_ag_line
state,,,,,
В бездействии,0,0,0,0,0
В консервации,0,0,0,0,0
В ликвидации,0,0,0,0,0
В накоплении/под циклической закачкой,0,0,0,0,0
В ожидании освоения прошлых лет,0,0,0,0,0
В освоении,5,5,0,5,5
В работе,112,112,0,112,104
ВНР,883,883,0,883,763
Наблюдательная,0,0,0,0,0


Заполним p_line из p_line_tm / p_ag_line

In [ ]:
(good_merged_df.loc[
    (good_merged_df['measure_date'] > '2023-12-20') 
    & (good_merged_df['well_name'] == '1529')]
    [['measure_date', 'well_name', 'state','p_line', 'p_line_tm', 'p_ag_line']]
    .head(20))

# Заполним p_line из p_line_tm / p_ag_line
good_merged_df['p_line'] = good_merged_df.apply(set_p_line, axis=1)

# Ищем пропущенные значения
print('Количество пропущенных значений в столбце p_intake_accel =',
      good_merged_df['p_line'].isna().sum()
)

##### Формирование сводной таблицы с дебитами и давлениями

Перед формированием сводной таблицы уберем все нулевые значения и аномально низкие значения дебитов.

В выгрузке из Wells дебиты протянуты во все года.

In [179]:
good_merged_df = good_merged_df.loc[
    (good_merged_df['q_fluid'] > 1.5)
    & (good_merged_df['q_oil'] > 1.5)
    & (good_merged_df['q_gaz'] > 5)
].replace(0, np.nan).reset_index(drop=True)

good_merged_df.head(2)

,measure_date,well_name,rig,character_x,exploitation_name,equipment_name,worktime,downtime,state,t_work_tm,f_rotat_tm,f_rotat,q_fluid,q_fluid_tm,q_oil,wcutting,q_gaz,q_gaz_tm,p_line,p_buff,p_buff_tm,p_bottomhole_research,p_intake_accel,p_intake_accel_tm,gas_factor,p_layer,p_annular,p_annular_tm,p_ag_line,field,p_line_tm,q_methanol_day,choke_d,p_bottom_from_intake,measure_date_new,month,year,bhp,layer_pres_calc,layer_pres_research,character_y
0,2024-04-01,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,NaN,В работе,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,18.6,NaN,79.879997,79.879997,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.1,NaN,NaN,86.57,2024-05-01,5,2024,91.7,110.3,NaN,Нефтяная
1,2024-04-02,4531,18.1,Нефтяная,Электропогружным насосом,МТ5А-125DP-1160,24.0,NaN,В работе,7.83,40.0,49.0,59.0,39.0,49.139999,2.25,34786.0,18124.0,18.9,18.6,17.5,NaN,79.879997,79.709999,707.940002,115.599998,33.0,107.07,17.34,Новопортовское,21.5,NaN,NaN,86.57,2024-05-02,5,2024,91.7,110.3,NaN,Нефтяная


In [181]:
# Создадим список столбцов с количественными значениями
num_cols = good_merged_df.select_dtypes('float').columns

# Создаем сводную таблицу для количественных значений
good_merged_df_pivot_num = good_merged_df.pivot_table(
    index=['well_name', 'year', 'month'],
    values=num_cols,
    aggfunc='mean'
    ).reset_index()

good_merged_df_pivot_num.head(2)

,well_name,year,month,bhp,choke_d,downtime,f_rotat,f_rotat_tm,gas_factor,layer_pres_calc,layer_pres_research,p_ag_line,p_annular,p_annular_tm,p_bottom_from_intake,p_bottomhole_research,p_buff,p_buff_tm,p_intake_accel,p_intake_accel_tm,p_layer,p_line,p_line_tm,q_fluid,q_fluid_tm,q_gaz,q_gaz_tm,q_oil,t_work_tm,wcutting,worktime
0,1003,2021,6,NaN,NaN,NaN,35.0,NaN,183.574814,NaN,NaN,34.933334,46.425926,NaN,146.179993,NaN,47.882593,NaN,138.771853,NaN,185.0,34.793333,NaN,129.777778,133.666672,19468.407407,20407.560547,105.667412,NaN,4.594445,24.0
1,1003,2021,7,143.5,NaN,NaN,35.0,NaN,286.333984,177.15,NaN,35.441666,52.033333,NaN,139.423004,NaN,49.300000,NaN,131.553334,NaN,185.0,35.833333,NaN,107.166667,110.266670,25392.333333,25500.283203,89.225670,NaN,1.000000,24.0


In [182]:
# Создадим список столбцов с категориальными значениями
str_cols = good_merged_df.select_dtypes(['datetime64', 'object']).columns
str_cols = list(str_cols.append(good_merged_df.select_dtypes('category').columns))
# Удалим из списка столбец 'well_name'
str_cols.remove('well_name')
str_cols

# Создаем сводную таблицу для категориальных значений
good_merged_df_pivot_str = good_merged_df.pivot_table(
    index=['well_name', 'year', 'month'],
    values=str_cols,
    aggfunc='first'
    ).reset_index()

good_merged_df_pivot_str.head(2)

,well_name,year,month,character_x,character_y,equipment_name,exploitation_name,field,measure_date,measure_date_new,rig,state
0,1003,2021,6,Нефтяная,None,МТ5А-125-2350,Электропогружным насосом,Новопортовское,2021-05-05,2021-06-05,21.00,ВНР
1,1003,2021,7,Нефтяная,Нефтяная,МТ5А-125-2350,Электропогружным насосом,Новопортовское,2021-06-01,2021-07-01,21.00,В работе


In [183]:
# Объединяем сводные таблицы с количественными и категориальными значениями
good_merged_df_pivot = good_merged_df_pivot_str.merge(
    good_merged_df_pivot_num, 
    on=['well_name', 'year', 'month'], 
    how='left').sort_values(by=['well_name', 'year', 'month'])

display(good_merged_df_pivot.head(2))
good_merged_df_pivot.info()

,well_name,year,month,character_x,character_y,equipment_name,exploitation_name,field,measure_date,measure_date_new,rig,state,bhp,choke_d,downtime,f_rotat,f_rotat_tm,gas_factor,layer_pres_calc,layer_pres_research,p_ag_line,p_annular,p_annular_tm,p_bottom_from_intake,p_bottomhole_research,p_buff,p_buff_tm,p_intake_accel,p_intake_accel_tm,p_layer,p_line,p_line_tm,q_fluid,q_fluid_tm,q_gaz,q_gaz_tm,q_oil,t_work_tm,wcutting,worktime
0,1003,2021,6,Нефтяная,None,МТ5А-125-2350,Электропогружным насосом,Новопортовское,2021-05-05,2021-06-05,21.00,ВНР,NaN,NaN,NaN,35.0,NaN,183.574814,NaN,NaN,34.933334,46.425926,NaN,146.179993,NaN,47.882593,NaN,138.771853,NaN,185.0,34.793333,NaN,129.777778,133.666672,19468.407407,20407.560547,105.667412,NaN,4.594445,24.0
1,1003,2021,7,Нефтяная,Нефтяная,МТ5А-125-2350,Электропогружным насосом,Новопортовское,2021-06-01,2021-07-01,21.00,В работе,143.5,NaN,NaN,35.0,NaN,286.333984,177.15,NaN,35.441666,52.033333,NaN,139.423004,NaN,49.300000,NaN,131.553334,NaN,185.0,35.833333,NaN,107.166667,110.266670,25392.333333,25500.283203,89.225670,NaN,1.000000,24.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 29274 entries, 0 to 29273
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   well_name              29274 non-null  object        
 1   year                   29274 non-null  int64         
 2   month                  29274 non-null  int64         
 3   character_x            29274 non-null  category      
 4   character_y            27529 non-null  object        
 5   equipment_name         29202 non-null  category      
 6   exploitation_name      29243 non-null  category      
 7   field                  29274 non-null  category      
 8   measure_date           29274 non-null  datetime64[ns]
 9   measure_date_new       29274 non-null  datetime64[ns]
 10  rig                    29070 non-null  category      
 11  state                  29274 non-null  category      
 12  bhp                    27529 non-null  float64       
 13  c

In [184]:
# Пересчитаем ГФ(м3/т)
good_merged_df_pivot['gas_factor'] = (good_merged_df_pivot['q_gaz']
                                      / good_merged_df_pivot['q_oil'])

# Формируем дату со сдвижкой по месяцу так, чтобы все данные приводились на первое число месяца
for idx, row in good_merged_df_pivot.iterrows():
    good_merged_df_pivot.loc[idx, 'measure_date'] = datetime.datetime(
    good_merged_df_pivot.loc[idx, 'year'], 
    good_merged_df_pivot.loc[idx, 'month'], 1
)
good_merged_df_pivot.head()

,well_name,year,month,character_x,character_y,equipment_name,exploitation_name,field,measure_date,measure_date_new,rig,state,bhp,choke_d,downtime,f_rotat,f_rotat_tm,gas_factor,layer_pres_calc,layer_pres_research,p_ag_line,p_annular,p_annular_tm,p_bottom_from_intake,p_bottomhole_research,p_buff,p_buff_tm,p_intake_accel,p_intake_accel_tm,p_layer,p_line,p_line_tm,q_fluid,q_fluid_tm,q_gaz,q_gaz_tm,q_oil,t_work_tm,wcutting,worktime
0,1003,2021,6,Нефтяная,None,МТ5А-125-2350,Электропогружным насосом,Новопортовское,2021-06-01,2021-06-05,21.00,ВНР,NaN,NaN,NaN,35.0,NaN,184.242304,NaN,NaN,34.933334,46.425926,NaN,146.179993,NaN,47.882593,NaN,138.771853,NaN,185.000000,34.793333,NaN,129.777778,133.666672,19468.407407,20407.560547,105.667412,NaN,4.594445,24.0
1,1003,2021,7,Нефтяная,Нефтяная,МТ5А-125-2350,Электропогружным насосом,Новопортовское,2021-07-01,2021-07-01,21.00,В работе,143.5,NaN,NaN,35.0,NaN,284.585516,177.15,NaN,35.441666,52.033333,NaN,139.423004,NaN,49.300000,NaN,131.553334,NaN,185.000000,35.833333,NaN,107.166667,110.266670,25392.333333,25500.283203,89.225670,NaN,1.000000,24.0
2,1003,2021,8,Нефтяная,Нефтяная,МТ5А-125-2350,Электропогружным насосом,Новопортовское,2021-08-01,2021-08-01,21.00,В работе,133.9,NaN,NaN,35.0,NaN,405.636391,171.10,NaN,37.512905,56.064518,NaN,131.219345,NaN,47.451613,NaN,123.258065,NaN,175.149994,37.516129,NaN,122.129032,122.032257,41246.677419,41462.875000,101.683868,NaN,1.000000,24.0
3,1003,2021,9,Нефтяная,Нефтяная,МТ5А-125-2350,Электропогружным насосом,Новопортовское,2021-09-01,2021-09-01,21.00,В работе,132.4,NaN,NaN,35.0,NaN,427.614493,167.59,NaN,38.904839,57.741936,NaN,128.986130,NaN,40.741935,NaN,120.638710,NaN,175.149994,39.161290,NaN,121.000000,120.161293,43079.677419,43503.183594,100.744194,NaN,1.000000,24.0
4,1003,2021,10,Нефтяная,Нефтяная,МТ5А-125-2350,Электропогружным насосом,Новопортовское,2021-10-01,2021-10-01,21.00,В работе,130.4,NaN,NaN,35.0,NaN,563.850989,167.00,NaN,39.060665,59.000000,NaN,126.993332,NaN,42.000000,NaN,120.000000,NaN,175.149994,40.000000,NaN,103.566667,103.433334,48038.033333,48005.367188,85.196327,NaN,2.200000,24.0


Вернем строки с Рпл_ГДИС, в которых Рзаб или дебиты = 0.

Это будут опорные точки для настройки Рпл в Prosper

In [198]:
all_pres_data_research = (all_pres_data_1[['well_name', 'measure_date', 'year', 'month', 
                            'layer_pres_calc', 'layer_pres_research']]
)

all_pres_data_research = (all_pres_data_research.groupby(by=['well_name', 'year', 'month'])
                          .agg({'measure_date': 'first', 
                                'layer_pres_calc': 'mean', 
                                'layer_pres_research': 'mean'})
                          ).query('layer_pres_research > 0')
all_pres_data_research.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4401 entries, ('1003', 2021, 5) to ('9929', 2023, 8)
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   measure_date         4401 non-null   datetime64[ns]
 1   layer_pres_calc      4104 non-null   float64       
 2   layer_pres_research  4401 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 126.9+ KB


In [189]:
# Для объединения датафреймов проиндексируем столбцы
good_merged_df_pivot.set_index(['well_name', 'year', 'month'], inplace=True)

# Определим список индексов, где есть Рпл ГДИС
added_index = []
for idx, row in all_pres_data_research.iterrows():
    # При этом проверяем, чтобы индексов с Рпл ГДИС не было в сводной табл с ШТР
    if idx not in good_merged_df_pivot.index:
        added_index.append(idx)

# Получим список индексов в формате Мультииндекса
added_index = pd.Index(added_index)
print(f'Количество индексов всего {len(added_index)}')
print(f'Количество уникальных значений индексов {len(added_index.unique())}')
added_index[:3]

Количество индексов всего 2039
Количество уникальных значений индексов 2039


MultiIndex([('1003', 2021, 5),
            ('1004', 2023, 5),
            ( '109', 1983, 7)],
           )

In [190]:
# Добавим в датафрейм строки с Рпл ГДИС, в которых другие параметры отсутствуют
good_merged_df_pivot = pd.concat([good_merged_df_pivot, all_pres_data_research.loc[added_index]])

# После добавления строк нужно отсортировать датафрейм по индексам
good_merged_df_pivot = good_merged_df_pivot.sort_index()
good_merged_df_pivot.shape

(31313, 37)

Переведем все давления в barA (абсолютное давление).

1 кгс/см2 = 1 atm g

1 barA = atm g * 1.01325 + 1

In [191]:
# В файле от НТЦ давления в кгс/см2 (как в отчетах ГДИ)
good_merged_df_pivot['bhp'] = good_merged_df_pivot['bhp'] * 1.01325 + 1
good_merged_df_pivot['layer_pres_calc'] = good_merged_df_pivot['layer_pres_calc'] * 1.01325 + 1
good_merged_df_pivot['layer_pres_research'] = good_merged_df_pivot['layer_pres_research'] * 1.01325 + 1

# В ШТР давления в атм технических или избыточных (atm g)
good_merged_df_pivot['p_ag_line'] = good_merged_df_pivot['p_ag_line'] * 1.01325 + 1
good_merged_df_pivot['p_annular'] = good_merged_df_pivot['p_annular'] * 1.01325 + 1
good_merged_df_pivot['p_annular_tm'] = good_merged_df_pivot['p_annular_tm'] * 1.01325 + 1
good_merged_df_pivot['p_bottomhole_research'] = good_merged_df_pivot['p_bottomhole_research'] * 1.01325 + 1
good_merged_df_pivot['p_buff'] = good_merged_df_pivot['p_buff'] * 1.01325 + 1
good_merged_df_pivot['p_buff_tm'] = good_merged_df_pivot['p_buff_tm'] * 1.01325 + 1
good_merged_df_pivot['p_intake_accel'] = good_merged_df_pivot['p_intake_accel'] * 1.01325 + 1
good_merged_df_pivot['p_intake_accel_tm'] = good_merged_df_pivot['p_intake_accel_tm'] * 1.01325 + 1
good_merged_df_pivot['p_layer'] = good_merged_df_pivot['p_layer'] * 1.01325 + 1
good_merged_df_pivot['p_line'] = good_merged_df_pivot['p_line'] * 1.01325 + 1
good_merged_df_pivot['p_line_tm'] = good_merged_df_pivot['p_line_tm'] * 1.01325 + 1

In [192]:
# Создадим столбец с количеством значений Рпл по ГДИС для каждой скв. (планируется использовать для фильтра)
layer_pres_research_count = good_merged_df_pivot.pivot_table(index='well_name',
                                 values='layer_pres_research',
                                 aggfunc='count').reset_index()
layer_pres_research_count.columns = ['well_name','layer_pres_research_count']
layer_pres_research_count

,well_name,layer_pres_research_count
0,1003,3
1,1004,1
2,109,1
3,1101,1
4,1102,1
...,...,...
785,9923,1
786,9924,1
787,9925,1
788,9926,2


In [193]:
# Присоединим столбец с количеством значений Рпл по ГДИС к сводной таблице с количественными значениями
good_merged_df_pivot = good_merged_df_pivot.merge(layer_pres_research_count, on='well_name', how='left')

# Удалим индексы, чтобы выгрузить датафрейм
good_merged_df_pivot.reset_index(inplace=True)

In [195]:
# Проверяем есть ли все скв. из исходного датафрейма all_pres_data_1 
# (в нем уже есть некоторая предобработка) в итоговом good_merged_df_pivot
not_found_wells = get_not_found_wells(
        all_pres_data_1['well_name'].unique(), 
        good_merged_df_pivot['well_name'].unique()
)

if not_found_wells:
    print(f'Not found wells: {len(not_found_wells)}')
    print(not_found_wells)
else:
    print('success!')

Not found wells: 4
['147', '92', '8058', '166']


Количество скважин в исходном и итоговом датафрейме различается из-за того, что были удалены пьезометрические скв.

In [ ]:
# Количество скв. в итоговом датафрейме
len(good_merged_df_pivot['well_name'].unique())

790

In [ ]:
# выгрузка со столбцами на англ. яз.
good_merged_df_pivot.to_csv('pivot_table_shtr_pres_en.csv', sep=';', encoding='cp1251', index=False)